In [1]:
import requests
from bs4 import BeautifulSoup
import json

In [2]:
response = requests.get('https://web6.seattle.gov/fas/registration/ResultsInteractive.aspx?Name=LLC&State=True')
response.status_code

200

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
table = soup.find('table',class_='Results')

In [5]:
count = 0
with open('data.jsonl', 'w') as f:
    for i in table.find_all('tr')[3:]:
        
        td_legal = i.find('td',class_='Legal') 
        a_legal = td_legal.find('a',class_='Legal') if td_legal else None
        td_dba = i.find('td',class_='Dba')
        td_contact = i.find('td',class_='Contact')
        td_phone = i.find('td',class_='Phone')
        td_email = i.find('td',class_='Email') 
        emails = None
        if td_email:
            if td_email.input:
                if td_email.input['email']:
                    emails = td_email.input['email']     
                
        td_WMBE = i.find('td',class_='Wmbe')
        if td_WMBE and td_WMBE.find('span'):
            wmbe_ethnicity = td_WMBE.span.text.split('-')
            wmbe = wmbe_ethnicity[0].strip() if len(wmbe_ethnicity) > 0 else ''
            ethnicity = wmbe_ethnicity[1].strip() if len(wmbe_ethnicity) > 1 and wmbe_ethnicity[1].strip() else None
        
        # print(f"Legal Name: {a_legal.text.strip()}")
        # print(f"DBA: {td_dba.text.strip()}")
        # print(f"Contact: {td_contact.text.strip()}")
        # print(f"Phone: {td_phone.text.strip()}")
        # print(f"Email: {emails}")
        # print(f"WMBE: {wmbe}")
        # print(f'Ethnicity: {ethnicity}')
        # print('-' * 40)
        
        rows ={
            'count': count,
            'Legal Name': a_legal.text.strip() if a_legal else '',
            'DBA': td_dba.text.strip() if td_dba else '',
            'Contact': td_contact.text.strip() if td_contact else '',
            'Phone': td_phone.text.strip() if td_phone else '',
            'Email': emails if emails else '',
            'WMBE': wmbe if wmbe else '',
            'Ethnicity':ethnicity if ethnicity else None
        }
        
        
        f.write(json.dumps(rows) + '\n')
        count += 1
        